# V3: false two layers(two pins only on the same layer)

In [32]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/bench3.router'
netlist_file_path = '../benchmark/bench3.nl'
gridfile_path  = '../benchmark/bench3.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 100
size of layer1: (60, 60)
size of layer2: (60, 60)


In [33]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target 目前支持到同一层S-T对的情况，显示两层
plot_problem("../output/bench3_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [34]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    #print(path)
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [35]:
def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty):
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer and (prev_x != x or prev_y != y):  #v4，层变化的，加穿孔cost(woods5/22)
            cell_cost += bend_penalty
    return cell_cost

# modified by junjun
def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty):#5.22 把layer2——grid输入，计算cost加入条件判断(woods5/22)
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])

            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell

                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer}) ##v4加两个方向如果为layer==1 ，up，==2，down(woods5/22)

    return neighbors

In [36]:
def false_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty):#修改了函数名，把layer2——grid输入(woods5/22)
    routing_table = {}
    routing_table_layer1 = {}
    routing_table_layer2 = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
            #这里区分不同层的pin，if语句有点多(woods5/22)
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
    
    nets_layer1 = [net for net in nets if net['pin1']['layer']==1 and net['pin2']['layer']==1]
    nets_layer2 = [net for net in nets if net['pin1']['layer']==2 and net['pin2']['layer']==2]    
    
    for net in nets_layer1:#在V4版本中不能分开处理
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path,costs1 = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,path)
            routing_table_layer1[net_id] = path
    plot_path('../output/bench3_v3_layer1.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer1_grid==-1),path_dict=routing_table_layer1) 
    #此处要重写plot函数，实现一个函数，根据path标记和绘制grid1/2的图(woods5/22)
        
    for net in nets_layer2:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer2_grid[pin1['x']][pin1['y']] = 1
        layer2_grid[pin2['x']][pin2['y']] = 1
        path,costs2 = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty) 
        mark_path_on_grid(layer2_grid,path)
        routing_table_layer2[net_id] = path
    plot_path('../output/bench3_v3_layer2.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer2_grid==-1),path_dict=routing_table_layer2)
    
    costs1.update(costs2)
    routing_table_layer1.update(routing_table_layer2)    
    routing_table = dict(sorted(routing_table_layer1.items(),key=lambda x:x[0]))
    return routing_table,costs1

In [37]:
layer1_grid = layer1_grid_original
layer2_grid = layer2_grid_original
routing_table,costs=false_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty)
generate_output_file(filepath_out,net_num,routing_table)